# 累積和とその周辺

# はじめに

この文書では、累積和、及びその周辺のアルゴリズム・データ構造について解説を行う。

累積和は、その内容自体は非常に簡単なものだが、非常に様々な問題においてあちらこちらに顔を出すので、重要なものでもある。

まずは累積和の基本的な考え方、実際の構築の仕方と利用方法から話を始め、ちょうど累積和の逆ともいえる通称「いもす法」、座標圧縮との組み合わせ、二次元累積和などに話を広げていく予定となっている。

# 累積和の基本

## なぜ累積和を使うのか？

ある $N$ 個の整数の要素を持つ数列 $A: A_1, A_2, ... , A_N$ があるとする。例えば、具体的には次のようなものだ。

$
2, -5, 42, 9, -38, 24, 19, -57, 88, 67
$

こういった感じの数列中の、連続した一部の合計、つまり連続部分和を求めることを考える。具体的に、```sum_part()``` の形で関数を作成することにしよう。ややこしくならないように、0-indexedの半開放で定義しておく。すなわち、```sum_part(a, b)``` のとき、返ってくる値は部分和 $A_{a-1} + A_{a} + ... + A_{b-2} + A_{b-2}$ とする。

また、あらかじめ $A$ はリストとして ```A``` に保存されているものとする。

In [1]:
A = [2, -5, 42, 9, -38, 24, 19, -57, 88, 67]

さて、単純に要素を足し合わせるだけなので、もちろん次のように書ける。

In [2]:
def sum_part(a, b):
    return sum(A[a:b])

print(sum_part(3, 6))  # = 9 - 38 + 24

-5


一回こっきり部分和を計算するだけならば、この方法でなんの問題もない。ただ、何度もおなじ計算を要求されるようなケースでは、関数ごとに計算量が $O(N)$ となるこの方法では、問題がある。

例えば、回答として $A_1, A_1+A_2, A_1+A_2+A_3 ... $ を全て求めよ、などと言われた場合はどうだろうか？上記のやり方だと計算量は $O(N^2)$ となり、$N \le 10^4$ あたりから間に合わなくなってくる。

……いや、でも、その場合は前から順番に足していって、その都度答えを出力すればいいだけじゃない？と思ったことだろう。  
まさに、それこそが累積和である。

## 累積和を求める

というわけで、さっそく累積和を使ってみよう。

累積和自体は広い概念を指す言葉だが、競技プログラミングにおいて累積和を用いるというとき、多くの場合は、連続部分和を計算するために事前計算を行うことを指す。つまり、最初に次のようなものを計算する。

$
[0, A_1, A_1+A_2, A_1+A_2+A_3, ... , \sum_{i=1}^{N}A_i]
$

最初に 0 を入れているのは、後で都合がいいからだ。さっそく計算させてみよう。

In [7]:
N = len(A)
cum_sum = [0] * (N+1)  # 累積和を保存していく

tmp = 0  # 現在の累積和を管理

for i in range(N):
    tmp += A[i]
    cum_sum[i+1] = tmp

print(cum_sum)

[0, 2, -3, 39, 48, 10, 34, 53, -4, 84, 151]


ちなみに、実は Python には累積和を自動的に求めてくれる ```itertools``` ライブラリの関数 ```accumulate``` が存在している。使用するためには、もちろんインポートする必要がある。

```itertools.accumulate``` は、キーワード引数 ```func``` で行う計算を柔軟に変更したり、```initial``` で初期値を設定したりもできる便利な関数だ。さっきの例なら、こう書ける。

In [6]:
from itertools import accumulate
N = len(A)
cum_sum = accumulate(A, initial=0)
print(cum_sum)
print(list(cum_sum))

[0, 2, -3, 39, 48, 10, 34, 53, -4, 84, 151]


見ての通り返ってくるのはイテレータなので、添字を利用するならリストに変換する必要がある。

こちらを使用するほうが便利なのだが、個人的には自分で書くことが多い。

さて、いずれにせよ、こうして得られた数列がなぜ便利なのかというと、連続部分和を素早く計算できるからだ。

例えば、$A_3+A_4+A_5$ を計算したいとする。これは、$A_1+A_2+A_3+A_4+A_5$ から、$A_1+A_2$ を引いたものに等しくなる。この計算を一般化することにより、あらゆる連続部分和を $O(1)$ で計算できるようになる。

というわけで、最初に書いた ```sum_part()``` を書き直してみよう。

In [9]:
csum_A = [0] * (N+1)
tmp = 0
for i in range(N):
    tmp += A[i]
    csum_A[i+1] = tmp

def sum_part(a, b):
    return csum_A[b] - csum_A[a]

sum_part(3, 6)

-5

# いもす法